# Overview

This script compares the weights generated by xCDAT and CDAT. The
dataset being used is from the post-regridded bilinear dataset from v2.12.1
via CDAT.


Related comment: https://github.com/E3SM-Project/e3sm_diags/pull/940#issuecomment-2699193211

- xCDAT: `spatial.get_weights()`

  - computes the relative weight of each rectilinear grid cell, but doesn't do any masking.
  - Calculate weights, then normalize with the

- CDAT: `genutil.averager.area_weights()`
  - computes the relative weight of each rectilinear grid cell (I think?), but does masking.


In [2]:
import cdms2 as cd
import numpy as np
import xarray as xr
import xcdat as xc # noqa: F401
from genutil.averager import area_weights

BASE_PATH = "/lcrc/group/e3sm/public_html/cdat-migration-fy24/25-02-19-branch-940-xesmf-diffs-mask-fix-sst-rmse/prov"
VAR_KEY = "variable_16"

/home/ac.tvo/miniforge3/envs/940-xesmf-diffs-sst-rmse/lib/python3.10/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


### CDAT Weights


In [3]:
sst1_cd = cd.open(f"{BASE_PATH}/sst_test_cdat_bilinear.nc")[VAR_KEY][:]
weights_cd = area_weights(sst1_cd, "xy")

In [4]:
weights_cd

variable_5
masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20)

In [7]:
weights_cd.sum()

np.float64(0.6010135459661596)

In [8]:
weights_cd.mean()

np.float64(1.9887939972407664e-05)

### xCDAT Weights


In [5]:
# xCDAT Weights
ds1_xr = xr.open_dataset(f"{BASE_PATH}/sst_test_cdat_bilinear.nc")
ds1_xr = ds1_xr.bounds.add_missing_bounds()
weights_xc = ds1_xr.spatial.get_weights(["X", "Y"], data_var=VAR_KEY)

In [6]:
weights_xc

<xarray.DataArray 'lat_lon_wts' (lon: 360, lat: 180)> Size: 518kB
array([[0.0001523 , 0.00045687, 0.00076129, ..., 0.00076129, 0.00045687,
        0.0001523 ],
       [0.0001523 , 0.00045687, 0.00076129, ..., 0.00076129, 0.00045687,
        0.0001523 ],
       [0.0001523 , 0.00045687, 0.00076129, ..., 0.00076129, 0.00045687,
        0.0001523 ],
       ...,
       [0.0001523 , 0.00045687, 0.00076129, ..., 0.00076129, 0.00045687,
        0.0001523 ],
       [0.0001523 , 0.00045687, 0.00076129, ..., 0.00076129, 0.00045687,
        0.0001523 ],
       [0.0001523 , 0.00045687, 0.00076129, ..., 0.00076129, 0.00045687,
        0.0001523 ]])
Coordinates:
  * lon      (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat      (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5

In [9]:
weights_xc.sum()

<xarray.DataArray 'lat_lon_wts' ()> Size: 8B
array(720.)

In [10]:
weights_xc.mean()

<xarray.DataArray 'lat_lon_wts' ()> Size: 8B
array(0.01111111)